### Loading IFC data  
To explore the IFC model, we use the [IfcOpenShell](https://docs.ifcopenshell.org/ifcopenshell-python/installation.html) module, you will have to install it again when using a virtual environment. <br>


In [65]:
import ifcopenshell as ios

model = ios.open("../datasets/heartbreak_hotel.ifc")

### Iterating over the IfcStair instances 
We want to iterate over all stair instances and their properties and drop them into a list.<br> 
The same applies to relational object instances, as these define other objects that may be related to a stair flight, such as railings... 

In [66]:
ifcstairs = model.by_type('IfcStair')
relations = model.by_type('IfcRelAggregates')
print(str(len(ifcstairs))+' IfcStairs found')

3 IfcStairs found


In [67]:
ifcdoors = model.by_type('IfcDoor')
print(str(len(ifcdoors))+' Ifcdoors found')

18 Ifcdoors found


### Drilling down the IfcModel in search for the IfcStair properties
For convenience, we use a custom Python class that represents a stair concept. 

In [68]:
class Stair:
    def __init__(self, name, riserheight, threadlength, railings):
        self.name = name
        self.RiserHeight = riserheight
        self.ThreadLength = threadlength
        self.NumberOfRailings = railings

class Door:
    def __init__(self, name, width):
        self.name = name
        self.width = width


In [69]:
stair_list = []

for ifcstair in ifcstairs:
    if hasattr(ifcstair, "IsDefinedBy"):
        for rel in ifcstair.IsDefinedBy:
            if rel.is_a("IfcRelDefinesByProperties"):
                property_set = rel.RelatingPropertyDefinition
                if property_set.is_a("IfcPropertySet") and property_set.Name == "Pset_StairCommon":
                    if hasattr(property_set, "HasProperties"):
                        for property in property_set.HasProperties:
                            if property.is_a("IfcPropertySingleValue"):
                                if property.Name == "RiserHeight":
                                    riserheight = round(property.NominalValue.wrappedValue)
                                if property.Name == "TreadLength":
                                    threadlength = round(property.NominalValue.wrappedValue)
    numberofrailings=0
    for rel in relations:
        for related_element in rel.RelatedObjects:
            if related_element.is_a("IfcRailing"):
                if rel.RelatingObject.GlobalId == ifcstair.GlobalId:
                    numberofrailings +=1
    
    new_stair = Stair(ifcstair.Name, riserheight, threadlength, numberofrailings)
    stair_list.append(new_stair)
    
print(len(stair_list), "stairs created.")

3 stairs created.


In [70]:
door_list = []

for ifcdoor in ifcdoors:
    name = ""
    width = 0.0
    if hasattr(ifcdoor, "Name"):
        name = ifcdoor.Name
    if hasattr(ifcdoor, "OverallWidth"):
        width = ifcdoor.OverallWidth
    
    new_door = Door(name=name, width=width)
    door_list.append(new_door)

print(len(door_list), "doors created.")

18 doors created.


### Dumping the stair & door data
Finally, we write the list of stair & door objects to a json file. In an intermediate step, we transform the list into a dictionary. Note that the json module has to be imported first. 

In [71]:

import json

stairs_dict = [stair.__dict__ for stair in stair_list]
with open("stairs.json", "w") as json_file:
    json.dump(stairs_dict, json_file, indent=4)

doors_dict = [door.__dict__ for door in door_list]
with open("doors.json", "w") as json_file:
    json.dump(doors_dict, json_file, indent=4)

## Or even better... straight info LBD?

In [80]:
# Import needed components from rdflib
from rdflib import Graph , Literal , BNode , Namespace , RDF , RDFS , OWL , URIRef

# initiate triple store, i.e. Graph()
g = Graph()

# Add namespaces and prefixes for ontologies
g.bind("owl", OWL)
BOT = Namespace("https://w3id.org/bot#")
g.bind("bot", BOT)
BEO = Namespace("https://w3id.org/beo#")
g.bind("beo", BEO)
MEP = Namespace("https://pi.pauwel.be/voc/distributionelement#")
g.bind("mep", MEP)

# Add namespace and prefix for instance graph (ABox)
INST = Namespace("https://example.org/OurBuilding#")
g.bind("", INST) # bind to default empty prefix
g.bind("inst", INST) # bind to inst prefix

# Initiate ontology entity
s = URIRef("https://example.org/OurBuilding")
p = RDF.type
o = OWL.Ontology
g.add((s, p, o))


# Now add the instances
for stair in stair_list:
    s = INST[str(stair.name).replace(" ","")]
    p = RDF.type
    o = URIRef("beo:Stair")
    g.add((s, p, o))

    # s = INST[str(stair.name).replace(" ","")]
    # p = RDF.type
    # o = URIRef("beo:Stair")
    # g.add((s, p, o))
    # Stair(name, riserheight, threadlength, numberofrailings)


for door in door_list:
    door_name = str(door.name).replace(" ","")
    s = INST[door_name]
    p = RDF.type
    o = URIRef("beo:Door")
    g.add((s, p, o))
    g.add((s, RDFS.label, Literal(door.name)))
    
    # Width property & value
    # w = INST["width_of_" + door_name]
    # g.add((w, RDF.type, RDF.Property))
    # g.add((w, OWL.DatatypeProperty, Literal(door.width, datatype="real"))) # datatype? Length?
    # # Link property to the object
    # g.add((s, OWL.ObjectProperty, w))

    # simple version
    g.add((s, URIRef("ex:width"), Literal(door.width)))


# Store the TTL
import os
g.serialize(destination = "house.ttl", format = "turtle")
print("Created house.ttl in folder:")
print(str(os.getcwd()))

Created house.ttl in folder:
/Users/stefan/Repositories/FireBIM/SSolDAC2024/What-the-check/IFC Conversion


In [78]:
# Iterate over triples in store and print them out.
print("--- printing raw triples ---")
for s, p, o in g:
    print((s, p, o))

--- printing raw triples ---
(rdflib.term.URIRef('https://example.org/OurBuilding#M_Single-Flush:0915x2134mm:358185'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('beo:Door'))
(rdflib.term.URIRef('https://example.org/OurBuilding#M_Single-Flush:0915x2134mm:358185'), rdflib.term.URIRef('http://www.w3.org/2002/07/owl#ObjectProperty'), rdflib.term.URIRef('https://example.org/OurBuilding#width_of_M_Single-Flush:0915x2134mm:358185'))
(rdflib.term.URIRef('https://example.org/OurBuilding#PrecastStair:Stair:397996'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('beo:Stair'))
(rdflib.term.URIRef('https://example.org/OurBuilding#M_Single-Flush:0915x2134mm:358185'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'), rdflib.term.Literal('M_Single-Flush:0915 x 2134mm:358185'))
(rdflib.term.URIRef('https://example.org/OurBuilding#width_of_M_Single-Flush:0915x2134mm:358185'), rdflib.term.URIRef('h

In [ ]:
for stmt in g:
    print(stmt)

print()
print("We have the following number of statements:")
print(len(g))

(rdflib.term.URIRef('https://example.org/OurBuilding'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/2002/07/owl#Ontology'))

We have the following number of statements:
1


### Perform some queries

In [ ]:
# Exercise: Add your SPARQL query here to list all spaces. 

ourQuery = """
PREFIX bot: <https://w3id.org/bot#>
PREFIX beo: <https://w3id.org/beo#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?s
WHERE{ ?s rdf:type beo:Door }
"""

qres = g.query(ourQuery)

print( "Found these objects: " )
for row in qres:
    print(f"{row.s}")

Found these objects: 
